In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [5]:
os.getcwd()

'c:\\Users\\Soren\\Documents\\GitHub\\IntroSocial23\\Exam final\\Data processing'

In [6]:
file_path = Path.cwd()/"Nasdaq mkt. cap 50m+.csv"
cwd = Path.cwd()

In [7]:
df = pd.read_csv(file_path)

In [8]:
df = df[["Symbol", "Market Cap"]]
df = df.reset_index(drop = True)

In [9]:
df

,Symbol,Market Cap
0,AADI,1.456480e+08
1,AAL,1.037541e+10
2,AAOI,4.767921e+08
3,AAON,5.236064e+09
4,AAPL,2.805719e+12
...,...,...
2183,ZUMZ,3.625572e+08
2184,ZVRA,1.829617e+08
2185,ZYME,5.080219e+08
2186,ZYNE,7.243086e+07


In [10]:
dictionary = dict(zip(df["Symbol"],df["Market Cap"]))

In [13]:
data = pd.read_csv(cwd/'Final_df.csv', sep = ';')
data = data.drop("Unnamed: 0", axis = 1)
data["Market Cap"] = data["Ticker"].map(dictionary)

In [8]:
# data.to_csv("df_final_marketcap.csv")

In [14]:
data

,date,open,high,low,adjusted_close,volume,RSI,MFI,volatility,returns,prev_returns,returns_pred,Ticker,LLM_score_sum,LLM_score_mean,Market Cap
0,2023-06-02,8.01,8.0900,7.8700,8.0600,74700,66.502362,48.430352,0.029908,2.025316,-1.250000,0.868486,AADI,0.0,0.0,1.456480e+08
1,2023-06-02,14.90,15.0700,14.6400,14.8700,31041300,23.119364,35.726831,0.014020,1.156463,-0.541272,-0.403497,AAL,0.0,0.0,1.037541e+10
2,2023-06-02,2.21,2.2700,2.1600,2.2700,148200,26.678504,47.236530,0.089979,2.714932,-0.450450,2.643172,AAOI,0.0,0.0,4.767921e+08
3,2023-06-02,88.82,91.6800,88.7300,91.5033,243389,36.479318,50.327996,0.016983,4.459876,1.223862,-0.458563,AAON,0.0,0.0,5.236064e+09
4,2023-06-02,181.03,181.7800,179.2600,180.7060,61945900,36.976220,45.237000,0.009638,0.477572,1.602217,-0.757141,AAPL,0.0,0.0,2.805719e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494863,2022-06-01,33.26,33.3400,32.2600,32.9700,557900,68.832023,57.020542,0.033060,0.487656,-3.300914,3.639672,ZUMZ,-1.0,-1.0,3.625572e+08
494864,2022-06-01,4.64,4.7050,4.4900,4.5700,257036,45.655482,44.451853,0.035933,-1.295896,-1.279318,3.501094,ZVRA,0.0,0.0,1.829617e+08
494865,2022-06-01,6.85,7.1100,6.6450,6.9800,996500,59.232450,56.263023,0.052836,3.869048,-2.183406,-3.295129,ZYME,0.0,0.0,5.080219e+08
494866,2022-06-01,0.97,0.9899,0.9021,0.9050,237207,39.897928,87.930630,0.087510,-5.483029,-1.875384,7.182320,ZYNE,0.0,0.0,7.243086e+07


In [17]:
# Import df_all_news_from_API.csv

import pandas as pd

df_all_news_from_API = pd.read_csv(cwd/'df_all_news_final.csv', sep = ';')

In [18]:
# df_all_news_from_API

# Merge data and df_all_news_from_API data using Ticker as key
# data_merged = pd.merge(df_all_news_from_API,data, left_on = 'ticker', right_on = 'Ticker', how = "left")
df_all_news_from_API["Market Cap"] = df_all_news_from_API["ticker"].map(dictionary)


In [22]:
data_merged = df_all_news_from_API

# Filter all tickers with 1000 news, since the maximum number of news from EODHD is 1000 per ticker
# Which would give a period with no news for the ticker
df_filter = data_merged.groupby("ticker")['title'].count().sort_values(ascending = False).reset_index()
df_filter = df_filter[(df_filter['title'] > 999)]
tickers = df_filter['ticker']

tickers


,ticker,title
22,AAPL,1000
23,UAL,1000
24,ADBE,966
25,CSCO,962
26,JBLU,908
27,VRTX,894
28,CRWD,894
29,AVGO,872
30,ROKU,858
31,ENPH,835


In [23]:
# datetime
data['date'] = pd.to_datetime(data['date'])


# For while to print plots of each ticker to see for how long the news are available
# If they only are missing 1 month they will still be included in the analysis
data1 = data.copy()

for ticker in tickers:
    data = data1[data1['Ticker'] == ticker]
    
    plt.figure(figsize = (20,10))
    plt.plot(data['date'],data['LLM_score_sum'], label = ticker)
    plt.legend()
    plt.savefig(ticker + '.png')
    plt.close()

